# 机器学习

## 决策树

![标签类别对照](./img.png)

![香农熵公式](./img_1.png)

```python


```

In [10]:
from math import log
import operator
from matplotlib import pyplot as plt


# jupyterNotebook模式下Debugger disconnected就是程序有Exception  无法在Exception处中断
def createDataSet():
    dataSet = [[0, 0, 0, 0, 'no'],  #数据集创建
               [0, 0, 0, 1, 'no'],
               [0, 1, 0, 1, 'yes'],
               [0, 1, 1, 0, 'yes'],
               [0, 0, 0, 0, 'no'],
               [1, 0, 0, 0, 'no'],
               [1, 0, 0, 1, 'no'],
               [1, 1, 1, 1, 'yes'],
               [1, 0, 1, 2, 'yes'],
               [1, 0, 1, 2, 'yes'],
               [2, 0, 1, 2, 'yes'],
               [2, 0, 1, 1, 'yes'],
               [2, 1, 0, 1, 'yes'],
               [2, 1, 0, 2, 'yes'],
               [2, 0, 0, 0, 'no']]
    labels = ['年龄', '有工作', '有自己的房子', '信贷情况']  #分类属性
    return dataSet, labels  #返回数据集和分类属性


# 求香农熵
def findentropy(dataset):
    featuredict = {}
    for data in dataset:
        if data[-1] not in featuredict:  # in for都是在字典键中遍历
            featuredict[data[-1]] = 1
        else:
            featuredict[data[-1]] += 1
    entropy = 0
    for feature in featuredict:
        rate = featuredict[feature] / len(dataset)
        entropy -= rate * log(rate, 2)
    return entropy


"""
函数说明:按照给定特征划分数据集

Parameters:
    dataSet - 待划分的数据集
    axis - 划分数据集的特征
    value - 需要返回的特征的值
Returns:
    无
Author:
    Jack Cui
Modify:
    2017-03-30
"""


# 子集  即条件经验熵种的“条件”
def splitDataSet(dataSet, axis, value):  # axis代表用哪个特征，value代表该特征对应的某一类别，比如年龄axis = 0，老年 value = 1
    retDataSet = []  #创建返回的数据集列表
    for featVec in dataSet:  #遍历数据集
        if featVec[axis] == value:
            reducedFeatVec = featVec[:axis]  #去掉axis特征    ⭐（为后续递归建树做准备，剔除已经用过的特征）
            reducedFeatVec.extend(featVec[axis + 1:])  #将符合条件的添加到返回的数据集
            retDataSet.append(reducedFeatVec)
    return retDataSet  #返回划分后的数据集


"""
函数说明:选择最优特征

Parameters:
    dataSet - 数据集
Returns:
    bestFeature - 信息增益最大的(最优)特征的索引值
Author:
    Jack Cui
Modify:
    2017-03-30
"""


def chooseBestFeatureToSplit(dataSet):
    numFeatures = len(dataSet[0]) - 1  #特征数量   排除最后一个分类结果
    baseEntropy = findentropy(dataSet)  #计算数据集的香农熵
    bestInfoGain = 0.0  #信息增益
    bestFeature = -1  #最优特征的索引值
    for i in range(numFeatures):  #遍历所有特征
        #获取dataSet的第i个所有特征
        featList = [example[i] for example in dataSet]
        uniqueVals = set(featList)  #创建set集合{},元素不可重复
        newEntropy = 0.0  #经验条件熵
        for value in uniqueVals:  #计算信息增益
            subDataSet = splitDataSet(dataSet, i, value)  #subDataSet划分后的子集
            prob = len(subDataSet) / float(len(dataSet))  #计算子集的概率
            newEntropy += prob * findentropy(subDataSet)  #根据公式计算经验条件熵
        infoGain = baseEntropy - newEntropy  #信息增益
        print("第%d个特征的增益为%.3f" % (i, infoGain))  #打印每个特征的信息增益
        if (infoGain > bestInfoGain):  #计算信息增益
            bestInfoGain = infoGain  #更新信息增益，找到最大的信息增益
            bestFeature = i  #记录信息增益最大的特征的索引值
    return bestFeature  #返回信息增益最大的特征的索引值


# def setuptree(dataset):
#     if len(dataset[0]) == 1:   # 递归返回 仅剩最终类别
#         return
#     bestfeature = chooseBestFeatureToSplit(dataset)
#     featureclass = [data[bestfeature] for data in dataset]   # bestfeature特征的所有类别
#     featureclass = list(set(featureclass))  # 去重
#     for i in featureclass:
#         subdataset = splitDataSet(dataSet=dataset, axis=chooseBestFeatureToSplit(dataset), value=i)
#         setuptree(subdataset)

"""
函数说明:统计classList中出现此处最多的元素(类标签)

Parameters:
    classList - 类标签列表
Returns:
    sortedClassCount[0][0] - 出现此处最多的元素(类标签)
Author:
    Jack Cui
Blog:
    http://blog.csdn.net/c406495762
Modify:
    2017-07-24
"""


def majorityCnt(classList):
    classCount = {}
    for vote in classList:  #统计classList中每个元素出现的次数
        if vote not in classCount.keys():
            classCount[vote] = 0
        classCount[vote] += 1  # lambda x: x[1]
    sortedClassCount = sorted(classCount.items(), key=operator.itemgetter(1), reverse=True)  #根据字典的值降序排序
    return sortedClassCount[0][0]  #返回classList中出现次数最多的元素


"""
函数说明:创建决策树

Parameters:
    dataSet - 训练数据集
    labels - 分类属性标签
    featLabels - 存储选择的最优特征标签
Returns:
    myTree - 决策树
Author:
    Jack Cui
Blog:
    http://blog.csdn.net/c406495762
Modify:
    2017-07-25
"""


def createTree(dataSet, labels, featLabels):
    classList = [example[-1] for example in dataSet]  #取分类标签(是否放贷:yes or no)
    # 递归终止条件
    if classList.count(classList[0]) == len(classList):  #如果类别完全相同则停止继续划分
        return classList[0]
    if len(dataSet[0]) == 1 or len(labels) == 0:  #遍历完所有特征时返回出现次数最多的类标签
        return majorityCnt(classList)

    bestFeat = chooseBestFeatureToSplit(dataSet)  #选择最优特征
    bestFeatLabel = labels[bestFeat]  #最优特征的标签
    featLabels.append(bestFeatLabel)
    myTree = {bestFeatLabel: {}}  #根据最优特征的标签生成树
    del (labels[bestFeat])  #删除已经使用特征【标签】
    featValues = [example[bestFeat] for example in dataSet]  #得到训练集中所有最优特征的属性值
    uniqueVals = set(featValues)  #去掉重复的属性值
    for value in uniqueVals:  #遍历特征，创建决策树。
        subLabels = labels[:]
        myTree[bestFeatLabel][value] = createTree(splitDataSet(dataSet, bestFeat, value), subLabels, featLabels)
    return myTree  # 未达到终止条件时返回储存新枝干的字典


"""
函数说明:使用决策树分类

Parameters:
    inputTree - 已经生成的决策树
    featLabels - 存储选择的最优特征标签
    testVec - 测试数据列表，顺序对应最优特征标签
Returns:
    classLabel - 分类结果
Author:
    Jack Cui
Blog:
    http://blog.csdn.net/c406495762
Modify:
    2017-07-25
"""


def classify(inputTree: dict, featLabels: list, testVec):  # testVecd测试数据
    firstStr = next(iter(inputTree))  # 取决策树第一个节点
    secondDict = inputTree[firstStr]  #下一个字典
    featIndex = featLabels.index(firstStr)  # 该节点名在标签集中的索引
    for key in secondDict.keys():
        if testVec[featIndex] == key:
            if type(secondDict[key]).__name__ == 'dict':    # 是子字典就继续递归
                classLabel = classify(secondDict[key], featLabels, testVec)
            else:   # 到达叶子节点
                classLabel = secondDict[key]
    try:
        return classLabel
    except NameError:
        return None

if __name__ == '__main__':
    decisiontree = {}
    dataset, labels = createDataSet()
    print(f'最优特征索引：{chooseBestFeatureToSplit(dataset)}')
    featlabels = []  # 记录分类结点名
    tree = createTree(dataset, labels, featlabels)
    print(tree)
    testVec = [0,1]  # 测试数据
    print(classify(tree, featlabels, testVec))


第0个特征的增益为0.083
第1个特征的增益为0.324
第2个特征的增益为0.420
第3个特征的增益为0.363
最优特征索引：2
第0个特征的增益为0.083
第1个特征的增益为0.324
第2个特征的增益为0.420
第3个特征的增益为0.363
第0个特征的增益为0.252
第1个特征的增益为0.918
第2个特征的增益为0.474
{'有自己的房子': {0: {'有工作': {0: 'no', 1: 'yes'}}, 1: 'yes'}}


KeyboardInterrupt: 